# 🐛 Google Colab上でのYOLOv8昆虫検出トレーニング

**プロジェクト**: 昆虫検出トレーニングプロジェクト  
**目的**: Roboflowデータセットを使用してカブトムシ検出のためのカスタムYOLOv8モデルを訓練  
**環境**: GPU加速付きGoogle Colaboratory  

---

## 📋 概要

このノートブックはYOLOv8昆虫検出モデルのためのインタラクティブトレーニングパイプラインを提供します。含まれる機能:

- ✅ 自動化された環境セットアップ
- ✅ GPU設定と検証
- ✅ データセットの準備と検証
- ✅ インタラクティブなモデルトレーニング
- ✅ リアルタイム進捗監視
- ✅ モデル評価とエクスポート
- ✅ 結果の可視化

---

## ⚡ クイックスタート

1. **GPUを有効化**: ランタイム → ランタイムタイプの変更 → GPUを選択
2. **すべてのセルを実行**: ランタイム → すべてのセルを実行
3. **データセットをアップロード**: プロンプトに従ってデータセットをアップロード
4. **トレーニングを監視**: リアルタイムトレーニング進捗を監視
5. **結果をダウンロード**: 訓練済みモデルをGoogle Driveに保存

---

## 🛠️ ステップ1: 環境セットアップとライブラリのインストール

In [ ]:
# 必要なライブラリのインストール
print("🔧 必要なライブラリをインストール中...")

!pip install ultralytics roboflow supervision
!pip install --upgrade torch torchvision

print("✅ インストールが完了しました！")

In [ ]:
# 必要なライブラリのインポート
import os
import sys
import time
import shutil
import zipfile
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 深層学習ライブラリ
import torch
import torchvision
from ultralytics import YOLO

# データ操作と可視化
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image, display, HTML, clear_output
import cv2

# Google Colab専用
from google.colab import files, drive
import yaml

print("📚 ライブラリのインポートが完了しました！")
print(f"🐍 Pythonバージョン: {sys.version}")
print(f"🔥 PyTorchバージョン: {torch.__version__}")
print(f"👁️ OpenCVバージョン: {cv2.__version__}")

## 🚀 ステップ2: GPU設定とシステム検証

In [ ]:
# GPU可用性と設定の確認
def check_gpu_setup():
    """GPU設定をチェックし、利用可能な場合はデバイスを設定する"""
    print("🔍 GPU設定を確認中...")
    print("="*50)
    
    # CUDA可用性の確認
    cuda_available = torch.cuda.is_available()
    print(f"CUDA利用可能: {cuda_available}")
    
    if cuda_available:
        device_count = torch.cuda.device_count()
        print(f"GPU数: {device_count}")
        
        for i in range(device_count):
            gpu_name = torch.cuda.get_device_name(i)
            gpu_memory = torch.cuda.get_device_properties(i).total_memory / 1e9
            print(f"GPU {i}: {gpu_name} ({gpu_memory:.1f} GB)")
        
        # デバイスの設定
        device = torch.device('cuda:0')
        print(f"\n✅ 使用デバイス: {device}")
        
        # 簡単な演算でGPUをテスト
        test_tensor = torch.rand(1000, 1000).to(device)
        result = torch.mm(test_tensor, test_tensor.t())
        print("✅ GPUテスト操作が成功しました！")
        
    else:
        print("⚠️ GPUが利用できません。トレーニングはCPUを使用します（低速）。")
        device = torch.device('cpu')
    
    print("="*50)
    return device

# GPU確認の実行
training_device = check_gpu_setup()

In [ ]:
# システム情報の表示
def display_system_info():
    """システム情報を表示する"""
    print("💻 システム情報")
    print("="*40)
    
    # CPU情報
    print(f"CPUコア数: {os.cpu_count()}")
    
    # メモリ情報（概算）
    import psutil
    memory = psutil.virtual_memory()
    print(f"RAM: {memory.total / 1e9:.1f} GB （利用可能: {memory.available / 1e9:.1f} GB）")
    
    # ディスク容量
    disk = psutil.disk_usage('/')
    print(f"ディスク: {disk.total / 1e9:.1f} GB （空き: {disk.free / 1e9:.1f} GB）")
    
    print("\n🔧 ソフトウェアバージョン")
    print("="*40)
    print(f"Python: {sys.version.split()[0]}")
    print(f"PyTorch: {torch.__version__}")
    print(f"Torchvision: {torchvision.__version__}")
    print(f"OpenCV: {cv2.__version__}")
    print(f"NumPy: {np.__version__}")
    
display_system_info()

## 📁 ステップ3: Google Drive連携とデータセットセットアップ

In [ ]:
# Google Driveをマウント
print("📁 Google Driveをマウント中...")
drive.mount('/content/drive')

# Google Drive内にプロジェクトディレクトリを作成
project_dir = Path('/content/drive/MyDrive/insect_detection_training')
project_dir.mkdir(exist_ok=True)

# サブディレクトリを作成
(project_dir / 'datasets').mkdir(exist_ok=True)
(project_dir / 'models').mkdir(exist_ok=True)
(project_dir / 'results').mkdir(exist_ok=True)
(project_dir / 'logs').mkdir(exist_ok=True)

print(f"✅ プロジェクトディレクトリを作成しました: {project_dir}")
print(f"📂 作業ディレクトリ: {os.getcwd()}")

# 作業ディレクトリを設定
os.chdir('/content')
print(f"📁 作業ディレクトリを変更しました: {os.getcwd()}")

## 📊 ステップ4: データセットの準備とアップロード

### オプションA: ローカルコンピューターからアップロード

In [ ]:
# オプションA: ローカルコンピューターからデータセットをアップロード
def upload_dataset_local():
    """ローカルコンピューターからデータセットをアップロードする"""
    print("📤 データセットのZIPファイルをアップロードしてください")
    print("ZIP内の期待される構造:")
    print("""
    dataset.zip
    ├── train/
    │   ├── images/
    │   └── labels/
    ├── valid/
    │   ├── images/
    │   └── labels/
    ├── test/
    │   ├── images/
    │   └── labels/
    └── data.yaml
    """)
    
    uploaded = files.upload()
    
    # アップロードされたファイルを展開
    for filename in uploaded.keys():
        if filename.endswith('.zip'):
            print(f"📦 {filename}を展開中...")
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                zip_ref.extractall('datasets')
            print("✅ データセットの展開が完了しました！")
            break
    else:
        print("❌ ZIPファイルが見つかりません。データセットを含むZIPファイルをアップロードしてください。")
        return False
    
    return True

# データセットをアップロード
upload_success = upload_dataset_local()

### オプションB: Roboflowからダウンロード（推奨）

In [ ]:
# オプションB: Roboflowからダウンロード
def download_roboflow_dataset():
    """Roboflowからカブトムシデータセットをダウンロードする"""
    print("🤖 Roboflowからカブトムシデータセットをダウンロード中...")
    
    try:
        from roboflow import Roboflow
        
        # Roboflowの初期化（APIキーの設定が必要な場合があります）
        # APIキーの取得先: https://app.roboflow.com/settings/api
        print("🔑 Roboflow APIキーを入力してください（スキップする場合はEnterを押してください）:")
        api_key = input("APIキー: ").strip()
        
        if api_key:
            rf = Roboflow(api_key=api_key)
            project = rf.workspace("z-algae-bilby").project("beetle")
            dataset = project.version(1).download("yolov8", location="datasets")
            print("✅ Roboflowからデータセットをダウンロードしました！")
            return True
        else:
            print("⚠️ APIキーが提供されませんでした。以下から手動でダウンロードできます:")
            print("https://universe.roboflow.com/z-algae-bilby/beetle/dataset/1")
            return False
            
    except Exception as e:
        print(f"❌ Roboflowからのダウンロードエラー: {e}")
        print("💡 代替案: 手動でダウンロードしてオプションAでアップロードしてください")
        return False

# Roboflowからダウンロード
download_success = download_roboflow_dataset()

### オプションC: 手動データセットセットアップ（テスト用）

In [ ]:
# オプションC: テスト用サンプルデータセットの作成
def create_sample_dataset():
    """テスト用のサンプルデータセット構造を作成する"""
    print("🧪 テスト用のサンプルデータセット構造を作成中...")
    
    # ディレクトリ構造の作成
    base_dir = Path('datasets')
    for split in ['train', 'valid', 'test']:
        (base_dir / split / 'images').mkdir(parents=True, exist_ok=True)
        (base_dir / split / 'labels').mkdir(parents=True, exist_ok=True)
    
    # サンプルdata.yamlの作成
    data_yaml = {
        'train': './train/images',
        'val': './valid/images', 
        'test': './test/images',
        'nc': 1,
        'names': ['beetle'],
        'roboflow': {
            'workspace': 'z-algae-bilby',
            'project': 'beetle',
            'version': 1,
            'license': 'CC BY 4.0',
            'url': 'https://universe.roboflow.com/z-algae-bilby/beetle/dataset/1'
        }
    }
    
    with open(base_dir / 'data.yaml', 'w') as f:
        yaml.dump(data_yaml, f, default_flow_style=False)
    
    print("✅ サンプルデータセット構造を作成しました！")
    print("⚠️ 注意: これは構造のみです。実際の画像とラベルを追加する必要があります。")
    return True

# サンプル構造を作成するには以下の行のコメントを外してください
# create_sample_dataset()

## ✅ ステップ5: データセットの検証と解析

In [ ]:
# データセット構造と内容の検証
def validate_dataset(dataset_path='datasets'):
    """
    データセットの構造と内容を検証する関数
    
    Args:
        dataset_path: データセットが格納されているディレクトリパス
    
    Returns:
        validation_success: 検証が成功したかのブール値
        dataset_config: data.yamlから読み込んだ設定情報
        dataset_stats: 各分割（train/valid/test）の統計情報
    """
    print("🔍 データセット構造を検証中...")
    print("="*50)
    
    # データセットディレクトリのPathオブジェクトを作成
    dataset_dir = Path(dataset_path)
    
    # data.yamlファイルの存在確認（YOLO形式では必須）
    data_yaml_path = dataset_dir / 'data.yaml'
    if not data_yaml_path.exists():
        print("❌ data.yaml not found!")
        return False
    
    # YAML設定ファイルを読み込み、データセット情報を取得
    with open(data_yaml_path, 'r') as f:
        data_config = yaml.safe_load(f)
    
    # データセット設定情報の表示（roboflow情報は除く）
    print("📄 データセット設定 (data.yaml):")
    for key, value in data_config.items():
        if key != 'roboflow':  # Roboflow固有の情報は表示をスキップ
            print(f"  {key}: {value}")
    
    # 各分割（train/valid/test）のディレクトリ構造と画像・ラベル数をチェック
    results = {}
    for split in ['train', 'valid', 'test']:
        # 画像とラベルのディレクトリパスを構築
        images_dir = dataset_dir / split / 'images'
        labels_dir = dataset_dir / split / 'labels'
        
        # ディレクトリが存在するかチェック
        if images_dir.exists() and labels_dir.exists():
            # 画像ファイルを検索（.jpg, .png, .jpeg拡張子）
            image_files = list(images_dir.glob('*.[jp][pn]g')) + list(images_dir.glob('*.jpeg'))
            # ラベルファイルを検索（.txt拡張子、YOLO形式）
            label_files = list(labels_dir.glob('*.txt'))
            
            # 統計情報を辞書に保存
            results[split] = {
                'images': len(image_files),
                'labels': len(label_files)
            }
            
            # 画像数とラベル数が一致しているかチェック
            # 一致していて、かつ0より大きい場合は正常
            status = "✅" if len(image_files) == len(label_files) and len(image_files) > 0 else "⚠️"
            print(f"{status} {split.upper()}: {len(image_files)} 画像, {len(label_files)} ラベル")
        else:
            # ディレクトリが見つからない場合
            print(f"❌ {split.upper()}: ディレクトリが見つかりません")
            results[split] = {'images': 0, 'labels': 0}
    
    # 全体の統計を計算
    total_images = sum(split['images'] for split in results.values())
    total_labels = sum(split['labels'] for split in results.values())
    
    print(f"\n📊 合計: {total_images} 画像, {total_labels} ラベル")
    
    # 検証成功の条件：画像が存在し、画像数とラベル数が一致
    if total_images > 0 and total_images == total_labels:
        print("✅ データセット検証が成功しました！")
        return True, data_config, results
    else:
        print("❌ データセット検証が失敗しました！")
        return False, None, None

# データセット検証を実行
validation_success, dataset_config, dataset_stats = validate_dataset()

In [ ]:
# データセット統計の可視化
def visualize_dataset_stats(stats):
    """
    データセットの統計情報をグラフで可視化する関数
    
    Args:
        stats: データセット統計情報の辞書
               形式: {'train': {'images': 数値, 'labels': 数値}, ...}
    """
    if not stats:
        print("❌ 表示するデータセット統計がありません")
        return
    
    print("📊 データセット統計の可視化")
    
    # matplotlib図を作成（1行2列のサブプロット）
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # プロット1: 分割ごとの画像数を棒グラフで表示
    splits = list(stats.keys())  # ['train', 'valid', 'test']
    image_counts = [stats[split]['images'] for split in splits]  # 各分割の画像数
    
    # カラーパレットを設定（視覚的に区別しやすい色）
    bars1 = ax1.bar(splits, image_counts, color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
    ax1.set_title('分割ごとの画像数', fontsize=14, fontweight='bold')
    ax1.set_ylabel('画像数')
    
    # 棒グラフの上に数値ラベルを追加
    for bar, count in zip(bars1, image_counts):
        # 棒の中央上部に数値を表示
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                str(count), ha='center', va='bottom', fontweight='bold')
    
    # プロット2: データセット分割の割合を円グラフで表示
    total = sum(image_counts)  # 全画像数
    percentages = [count/total*100 for count in image_counts]  # 各分割の割合
    
    # 円グラフを作成（分割名と画像数、割合を表示）
    ax2.pie(percentages, labels=[f'{split}\n({count} 画像)' for split, count in zip(splits, image_counts)], 
            autopct='%1.1f%%', startangle=90, colors=['#FF6B6B', '#4ECDC4', '#45B7D1'])
    ax2.set_title('データセット分割の割合', fontsize=14, fontweight='bold')
    
    # レイアウトを調整して表示
    plt.tight_layout()
    plt.show()
    
    # テキストでのサマリーも表示
    print(f"\n📈 データセットサマリー:")
    print(f"総画像数: {total}")
    for split, count in zip(splits, image_counts):
        percentage = count/total*100
        print(f"{split.upper()}: {count} 画像 ({percentage:.1f}%)")

# 検証が成功した場合のみ可視化を実行
if validation_success:
    visualize_dataset_stats(dataset_stats)
else:
    print("⚠️ データセットを可視化できません - 検証が失敗しました")

In [ ]:
# データセットからサンプル画像の表示
def display_sample_images(dataset_path='datasets', num_samples=6):
    """
    データセットからランダムにサンプル画像を選択して表示する関数
    
    Args:
        dataset_path: データセットのパス
        num_samples: 表示するサンプル画像数（デフォルト：6枚）
    """
    if not validation_success:
        print("⚠️ サンプルを表示できません - データセット検証が失敗しました")
        return
    
    print(f"🖼️ トレーニングセットから{num_samples}枚のサンプル画像を表示")
    
    # トレーニング画像ディレクトリのパスを構築
    dataset_dir = Path(dataset_path)
    train_images = list((dataset_dir / 'train' / 'images').glob('*.[jp][pn]g'))
    
    # 画像ファイルが見つからない場合のエラーハンドリング
    if len(train_images) == 0:
        print("❌ トレーニングセットに画像が見つかりません")
        return
    
    # 利用可能な画像数とリクエスト数の小さい方を選択
    # ランダムに画像をサンプリング（重複なし）
    sample_images = np.random.choice(train_images, min(num_samples, len(train_images)), replace=False)
    
    # サブプロットの配置を計算（3列固定）
    cols = 3
    rows = (num_samples + cols - 1) // cols  # 必要な行数を計算
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
    
    # 1行の場合は2次元配列に変換（統一的に扱うため）
    if rows == 1:
        axes = axes.reshape(1, -1)
    
    # 各サンプル画像を処理して表示
    for i, img_path in enumerate(sample_images):
        # サブプロットの位置を計算
        row = i // cols
        col = i % cols
        
        # OpenCVで画像を読み込み（BGR形式）
        img = cv2.imread(str(img_path))
        # matplotlibで表示するためにRGB形式に変換
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # 画像を表示
        axes[row, col].imshow(img_rgb)
        axes[row, col].set_title(f"サンプル {i+1}: {img_path.name}", fontsize=10)
        axes[row, col].axis('off')  # 軸を非表示
    
    # 使用されていないサブプロットを非表示にする
    for i in range(len(sample_images), rows * cols):
        row = i // cols
        col = i % cols
        axes[row, col].axis('off')
    
    # レイアウトを調整して表示
    plt.tight_layout()
    plt.show()

# サンプル画像の表示を実行
display_sample_images()

## 🎯 ステップ6: トレーニング設定とモデル選択

In [ ]:
# トレーニング設定クラス
class TrainingConfig:
    """
    YOLOv8モデルのトレーニングに必要な設定を管理するクラス
    
    このクラスはトレーニングのすべてのパラメータを一元管理し、
    GPU環境に応じた自動最適化も行います。
    """
    
    def __init__(self):
        # === モデル設定 ===
        # YOLOv8のモデルサイズ（n=nano, s=small, m=medium, l=large, x=extra-large）
        # 小さいモデルほど高速だが精度は低い、大きいモデルほど高精度だが計算コストが高い
        self.model_size = 'n'  
        self.pretrained_model = f'yolov8{self.model_size}.pt'  # 事前学習済みモデルファイル名
        
        # === トレーニングパラメータ ===
        self.epochs = 100          # 訓練エポック数（データセット全体を何回学習するか）
        self.batch_size = 16       # バッチサイズ（一度に処理する画像数、GPU メモリに依存）
        self.image_size = 640      # 入力画像サイズ（YOLO標準は640x640ピクセル）
        self.device = 'auto'       # 使用デバイス（'auto'=自動選択, 'cpu', '0'=GPU0など）
        
        # === データ設定 ===
        self.data_yaml = 'datasets/data.yaml'  # データセット設定ファイルのパス
        
        # === 出力設定 ===
        self.project_name = 'training_results'        # プロジェクト名（結果保存用）
        self.experiment_name = 'beetle_detection_colab' # 実験名（この実行の識別子）
        
        # === 高度な設定 ===
        self.patience = 50      # 早期停止の忍耐値（何エポック改善がなければ停止するか）
        self.save_period = 10   # チェックポイント保存間隔（Nエポックごとに中間結果を保存）
        self.workers = 2        # データローダーのワーカー数（並列処理数）
        
        # === 最適化設定 ===
        self.optimizer = 'AdamW'    # 最適化アルゴリズム（SGD, Adam, AdamWから選択）
        self.lr0 = 0.01            # 初期学習率（学習の速度を制御）
        self.weight_decay = 0.0005  # 重み減衰（過学習を防ぐ正則化パラメータ）
        
    def display_config(self):
        """現在の設定を見やすい形式で表示する"""
        print("🎯 トレーニング設定")
        print("="*40)
        print(f"モデル: {self.pretrained_model}")
        print(f"エポック数: {self.epochs}")
        print(f"バッチサイズ: {self.batch_size}")
        print(f"画像サイズ: {self.image_size}")
        print(f"デバイス: {self.device}")
        print(f"データセット: {self.data_yaml}")
        print(f"プロジェクト: {self.project_name}/{self.experiment_name}")
        print(f"最適化手法: {self.optimizer}")
        print(f"学習率: {self.lr0}")
        print(f"重み減衰: {self.weight_decay}")
        print("="*40)

# 設定インスタンスを作成
config = TrainingConfig()
config.display_config()

# GPU メモリに基づくバッチサイズの自動調整
if torch.cuda.is_available():
    # GPU メモリ容量を取得（GB単位）
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"\n🎮 GPU メモリ: {gpu_memory:.1f} GB")
    
    # メモリ容量に応じてバッチサイズを調整
    if gpu_memory < 8:
        # 8GB未満：メモリ不足を避けるため小さなバッチサイズ
        config.batch_size = 8
        print("⚡ GPU メモリ最適化のため、バッチサイズを8に削減しました")
    elif gpu_memory >= 16:
        # 16GB以上：大きなバッチサイズでGPUを効率活用
        config.batch_size = 32
        print("🚀 GPU利用効率向上のため、バッチサイズを32に増加しました")
    
print(f"\n📊 最終バッチサイズ: {config.batch_size}")

## 🚀 ステップ7: モデルトレーニング実行

In [ ]:
# 事前学習済みモデルの読み込み
def load_pretrained_model(model_name):
    """
    指定されたYOLOv8事前学習済みモデルを読み込む関数
    
    Args:
        model_name: モデルファイル名（例：'yolov8n.pt'）
    
    Returns:
        model: 読み込まれたYOLOモデルオブジェクト（成功時）またはNone（失敗時）
    """
    print(f"📥 事前学習済みモデルを読み込み中: {model_name}")
    
    try:
        # UltralyticsのYOLOクラスでモデルを読み込み
        # 初回実行時は自動的にインターネットからダウンロードされる
        model = YOLO(model_name)
        print(f"✅ モデルの読み込みが成功しました！")
        
        # モデル情報の表示
        print(f"\n📋 モデル情報:")
        print(f"モデルファイル: {model_name}")
        print(f"タスク: {model.task}")  # 'detect'（物体検出）が表示される
        
        return model
    
    except Exception as e:
        # エラーが発生した場合の処理
        print(f"❌ モデル読み込みエラー: {e}")
        return None

# 設定で指定されたモデルを読み込み
model = load_pretrained_model(config.pretrained_model)

In [ ]:
# モデル訓練の実行
def train_model(model, config):
    """
    YOLOv8モデルの訓練を実行する関数
    
    Args:
        model: 読み込み済みのYOLOv8モデルオブジェクト
        config: TrainingConfigクラスのインスタンス（訓練設定）
    
    Returns:
        results: 訓練結果オブジェクト（成功時）またはNone（失敗時）
    """
    # 前提条件のチェック
    if model is None:
        print("❌ 訓練を開始できません - モデルが読み込まれていません")
        return None
    
    if not validation_success:
        print("❌ 訓練を開始できません - データセット検証が失敗しました")
        return None
    
    # 訓練開始の通知
    print("🚀 モデル訓練を開始します...")
    print("⏱️ 設定によっては時間がかかる場合があります")
    print("📊 訓練の進捗は以下に表示されます")
    print("="*60)
    
    # 訓練開始時刻を記録（訓練時間計測のため）
    start_time = time.time()
    
    try:
        # YOLOv8のtrainメソッドを呼び出して訓練を実行
        # 各パラメータの説明：
        # - data: データセット設定ファイル（data.yaml）のパス
        # - epochs: 訓練エポック数（データセット全体を何回学習するか）
        # - batch: バッチサイズ（一度に処理する画像数）
        # - imgsz: 入力画像サイズ（リサイズ後のサイズ）
        # - device: 使用デバイス（GPU/CPU）
        # - project: 結果保存用のプロジェクト名
        # - name: この実験の名前（フォルダ名として使用）
        # - save: モデルの重みを保存するかどうか
        # - save_period: 中間結果の保存間隔
        # - patience: 早期停止の忍耐値（改善が見られないエポック数）
        # - workers: データローダーの並列処理数
        # - optimizer: 最適化アルゴリズム
        # - lr0: 初期学習率
        # - weight_decay: 重み減衰（正則化パラメータ）
        # - val: 検証を実行するかどうか
        # - plots: 結果グラフを生成するかどうか
        # - verbose: 詳細ログを表示するかどうか
        results = model.train(
            data=config.data_yaml,
            epochs=config.epochs,
            batch=config.batch_size,
            imgsz=config.image_size,
            device=config.device,
            project=config.project_name,
            name=config.experiment_name,
            save=True,
            save_period=config.save_period,
            patience=config.patience,
            workers=config.workers,
            optimizer=config.optimizer,
            lr0=config.lr0,
            weight_decay=config.weight_decay,
            val=True,
            plots=True,
            verbose=True
        )
        
        # 訓練時間の計算と表示
        training_time = time.time() - start_time
        hours = int(training_time // 3600)      # 時間
        minutes = int((training_time % 3600) // 60)  # 分
        seconds = int(training_time % 60)       # 秒
        
        # 訓練完了の通知
        print("="*60)
        print(f"✅ 訓練が正常に完了しました！")
        print(f"⏱️ 総訓練時間: {hours:02d}:{minutes:02d}:{seconds:02d}")
        print(f"📁 結果保存先: {config.project_name}/{config.experiment_name}")
        
        return results
        
    except Exception as e:
        # エラーが発生した場合の処理
        print(f"❌ 訓練が失敗しました: {e}")
        return None

# 訓練開始の警告（誤実行防止）
print("⚠️ 警告: 5秒後に訓練を開始します...")
time.sleep(5)

# モデル訓練を実行（これは時間がかかります！）
training_results = train_model(model, config)

## 📊 ステップ8: トレーニング結果の解析と可視化

In [ ]:
# Display training results
def display_training_results(results, config):
    if results is None:
        print("❌ No training results to display")
        return
    
    print("📊 Training Results Summary")
    print("="*50)
    
    # Results directory
    results_dir = Path(config.project_name) / config.experiment_name
    
    # Display key metrics if available
    if hasattr(results, 'results_dict'):
        metrics = results.results_dict
        print("🎯 Final Metrics:")
        for key, value in metrics.items():
            if isinstance(value, (int, float)):
                print(f"  {key}: {value:.4f}")
    
    # Check for results plots
    plots_to_show = [
        ('results.png', '📈 Training/Validation Curves'),
        ('confusion_matrix.png', '🎯 Confusion Matrix'),
        ('labels.jpg', '📊 Label Distribution'),
        ('val_batch0_pred.jpg', '🔍 Validation Predictions')
    ]
    
    for plot_file, title in plots_to_show:
        plot_path = results_dir / plot_file
        if plot_path.exists():
            print(f"\n{title}")
            display(Image(str(plot_path)))
        else:
            print(f"⚠️ {title} not found: {plot_path}")

# Display results
if training_results:
    display_training_results(training_results, config)
else:
    print("⚠️ No training results available to display")

In [ ]:
# 最良モデルの読み込みと検証実行
def validate_trained_model(config):
    """
    訓練完了後の最良モデルを読み込み、テストセットで検証を実行する関数
    
    Args:
        config: TrainingConfigインスタンス（結果の保存場所などの情報）
    
    Returns:
        best_model: 読み込まれた最良モデル（成功時）またはNone（失敗時）
        val_results: 検証結果オブジェクト（成功時）またはNone（失敗時）
    """
    print("🧪 最良モデルを読み込んで検証を実行中...")
    
    # 訓練で生成された最良モデルのパスを構築
    best_model_path = Path(config.project_name) / config.experiment_name / 'weights' / 'best.pt'
    
    # ファイルの存在確認
    if not best_model_path.exists():
        print(f"❌ 最良モデルが見つかりません: {best_model_path}")
        return None, None
    
    try:
        # 最良モデルを読み込み
        # best.ptは訓練中に最も良い性能を示したエポックのモデル
        best_model = YOLO(str(best_model_path))
        print(f"✅ 最良モデルを読み込みました: {best_model_path}")
        
        # テストセットで検証を実行
        print("\n🎯 テストセットで検証を実行中...")
        # val()メソッドはモデルの性能指標を計算
        val_results = best_model.val(data=config.data_yaml)
        
        # 検証結果の表示
        if hasattr(val_results, 'box'):
            box_metrics = val_results.box  # バウンディングボックス検出の指標
            print("\n📊 検証メトリクス:")
            # mAP@0.5: IoU閾値0.5での平均精度（最も重要な指標）
            print(f"  mAP@0.5: {box_metrics.map50:.4f}")
            # mAP@0.5:0.95: IoU閾値0.5〜0.95の平均での平均精度（より厳しい評価）
            print(f"  mAP@0.5:0.95: {box_metrics.map:.4f}")
            # Precision: 検出したもののうち正解の割合
            print(f"  Precision: {box_metrics.mp:.4f}")
            # Recall: 実際のオブジェクトのうち検出できた割合
            print(f"  Recall: {box_metrics.mr:.4f}")
        
        return best_model, val_results
        
    except Exception as e:
        print(f"❌ 検証が失敗しました: {e}")
        return None, None

# 訓練が成功した場合のみ検証を実行
if training_results:
    best_model, validation_results = validate_trained_model(config)
else:
    print("⚠️ 検証をスキップします - 訓練が完了していません")
    best_model, validation_results = None, None

## 🔍 ステップ9: モデル推論とテスト

In [ ]:
# モデル推論のテスト
def test_model_inference(model, config, num_samples=4):
    """
    訓練済みモデルを使用してサンプル画像で推論テストを実行する関数
    
    Args:
        model: 訓練済みのYOLOモデル
        config: 設定オブジェクト
        num_samples: テストするサンプル画像数（デフォルト：4枚）
    """
    if model is None:
        print("❌ テスト用のモデルがありません")
        return
    
    print(f"🔍 {num_samples}枚のサンプル画像でモデル推論をテスト中...")
    
    # テスト画像ディレクトリの取得（test→valid→trainの順で探索）
    test_images_dir = Path('datasets/test/images')
    if not test_images_dir.exists():
        # テストディレクトリがない場合は検証用ディレクトリを使用
        test_images_dir = Path('datasets/valid/images')
    
    if not test_images_dir.exists():
        print("❌ テスト画像が見つかりません")
        return
    
    # 画像ファイルを取得
    image_files = list(test_images_dir.glob('*.[jp][pn]g'))
    if len(image_files) == 0:
        print("❌ 画像ファイルが見つかりません")
        return
    
    # ランダムにサンプル画像を選択
    sample_images = np.random.choice(image_files, min(num_samples, len(image_files)), replace=False)
    
    # 2x2のサブプロットを作成
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.flatten()  # 2次元配列を1次元に変換
    
    for i, img_path in enumerate(sample_images):
        if i >= len(axes):
            break
        
        try:
            # モデルで推論を実行
            # results[0]は最初の画像の結果（バッチ処理の場合は複数）
            results = model(str(img_path))
            
            # 検出結果を画像に描画
            # plot()メソッドはバウンディングボックスとクラス名を描画
            annotated_img = results[0].plot()
            
            # OpenCVのBGR形式からmatplotlibのRGB形式に変換
            annotated_img_rgb = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)
            
            # 画像を表示
            axes[i].imshow(annotated_img_rgb)
            
            # 検出情報を取得してタイトルに表示
            detections = len(results[0].boxes) if results[0].boxes is not None else 0
            confidence = results[0].boxes.conf.max().item() if detections > 0 else 0
            
            axes[i].set_title(f"{img_path.name}\n検出数: {detections}, 最大信頼度: {confidence:.3f}", 
                            fontsize=10)
            axes[i].axis('off')  # 軸を非表示
            
        except Exception as e:
            print(f"❌ {img_path.name}の処理エラー: {e}")
            # エラーが発生した場合はエラーメッセージを表示
            axes[i].text(0.5, 0.5, f"エラー: {str(e)[:50]}...", 
                        ha='center', va='center', transform=axes[i].transAxes)
            axes[i].axis('off')
    
    # 使用していないサブプロットを非表示
    for i in range(len(sample_images), len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.suptitle('🔍 モデル推論結果', fontsize=16, fontweight='bold', y=1.02)
    plt.show()

# 最良モデルが利用可能な場合に推論テストを実行
if best_model:
    test_model_inference(best_model, config)
else:
    print("⚠️ 推論テストをスキップします - 訓練済みモデルがありません")

## 💾 ステップ10: モデルエクスポートとダウンロード

In [ ]:
# 様々な形式でのモデルエクスポート
def export_trained_model(model, config, formats=['onnx', 'torchscript']):
    """
    訓練済みモデルを様々な形式にエクスポートする関数
    
    Args:
        model: エクスポートするYOLOモデル
        config: 設定オブジェクト（画像サイズなどの情報）
        formats: エクスポートする形式のリスト（デフォルト：ONNX、TorchScript）
    
    Returns:
        exported_files: エクスポートされたファイルのパスリスト
    """
    if model is None:
        print("❌ エクスポート用のモデルがありません")
        return
    
    print(f"📦 モデルを以下の形式にエクスポート中: {formats}")
    
    exported_files = []
    
    for format_type in formats:
        try:
            print(f"\n🔄 {format_type.upper()}形式でエクスポート中...")
            
            # export()メソッドで指定形式にモデルをエクスポート
            # format: エクスポート形式（'onnx', 'torchscript', 'engine'など）
            # imgsz: エクスポート時の画像サイズ（推論時と同じサイズにする）
            export_path = model.export(format=format_type, imgsz=config.image_size)
            exported_files.append(export_path)
            print(f"✅ {format_type.upper()}エクスポート成功: {export_path}")
            
        except Exception as e:
            print(f"❌ {format_type.upper()}エクスポート失敗: {e}")
    
    # エクスポート結果のサマリー
    if exported_files:
        print(f"\n🎉 {len(exported_files)}個のモデル形式のエクスポートが成功しました")
        for file_path in exported_files:
            print(f"  📄 {file_path}")
    
    return exported_files

# 最良モデルが利用可能な場合にエクスポートを実行
if best_model:
    exported_models = export_trained_model(best_model, config)
else:
    print("⚠️ モデルエクスポートをスキップします - 訓練済みモデルがありません")
    exported_models = []

In [ ]:
# 訓練結果をGoogle Driveにコピー
def copy_results_to_drive(config):
    """
    訓練結果をGoogle Driveに永続保存する関数
    
    Args:
        config: 設定オブジェクト（プロジェクト名、実験名などの情報）
    
    Returns:
        bool: コピーが成功したかどうか
    """
    print("💾 訓練結果をGoogle Driveにコピー中...")
    
    # コピー元ディレクトリ（Colabの一時領域）
    source_dir = Path(config.project_name) / config.experiment_name
    
    # コピー先ディレクトリ（Google Drive内の永続領域）
    drive_dir = Path('/content/drive/MyDrive/insect_detection_training/results') / config.experiment_name
    
    # コピー元ディレクトリの存在確認
    if not source_dir.exists():
        print(f"❌ コピー元ディレクトリが見つかりません: {source_dir}")
        return False
    
    try:
        # Google Drive内に保存先ディレクトリを作成
        drive_dir.mkdir(parents=True, exist_ok=True)
        
        # ディレクトリ全体を再帰的にコピー
        # dirs_exist_ok=True: 既存ディレクトリがあっても上書き
        import shutil
        shutil.copytree(source_dir, drive_dir, dirs_exist_ok=True)
        
        print(f"✅ 結果をコピーしました: {drive_dir}")
        
        # 重要ファイルの確認と表示
        important_files = [
            'weights/best.pt',      # 最良モデルの重み
            'weights/last.pt',      # 最終エポックのモデル重み
            'results.png',          # 訓練結果グラフ
            'confusion_matrix.png'  # 混同行列
        ]
        
        print("\n📁 Google Drive内の重要ファイル:")
        for file_path in important_files:
            full_path = drive_dir / file_path
            if full_path.exists():
                # ファイルサイズをMB単位で表示
                size_mb = full_path.stat().st_size / (1024 * 1024)
                print(f"  ✅ {file_path} ({size_mb:.1f} MB)")
            else:
                print(f"  ❌ {file_path} (見つかりません)")
        
        return True
        
    except Exception as e:
        print(f"❌ Google Driveへのコピーエラー: {e}")
        return False

# 訓練結果が存在する場合にGoogle Driveへコピー
if training_results:
    copy_success = copy_results_to_drive(config)
else:
    print("⚠️ Google Driveへのコピーをスキップします - 訓練結果がありません")

In [ ]:
# Download trained models to local computer
def download_models(config):
    print("⬇️ Preparing model files for download...")
    
    results_dir = Path(config.project_name) / config.experiment_name
    weights_dir = results_dir / 'weights'
    
    if not weights_dir.exists():
        print(f"❌ Weights directory not found: {weights_dir}")
        return
    
    # Files to download
    download_files = {
        'best.pt': 'Best model weights',
        'last.pt': 'Last epoch weights'
    }
    
    print("\n📥 Available for download:")
    
    for filename, description in download_files.items():
        file_path = weights_dir / filename
        if file_path.exists():
            size_mb = file_path.stat().st_size / (1024 * 1024)
            print(f"  📄 {filename}: {description} ({size_mb:.1f} MB)")
            
            # Trigger download
            try:
                files.download(str(file_path))
                print(f"  ✅ {filename} download initiated")
            except Exception as e:
                print(f"  ❌ Error downloading {filename}: {e}")
        else:
            print(f"  ❌ {filename}: Not found")
    
    # Also download results plot
    results_plot = results_dir / 'results.png'
    if results_plot.exists():
        try:
            files.download(str(results_plot))
            print(f"  ✅ results.png download initiated")
        except Exception as e:
            print(f"  ❌ Error downloading results.png: {e}")

# Download models
if training_results:
    download_models(config)
else:
    print("⚠️ No models available for download")

## 📋 ステップ11: トレーニングサマリーと次のステップ

In [ ]:
# Generate training summary
def generate_training_summary(config, training_results, validation_results):
    print("📋 TRAINING SUMMARY")
    print("="*60)
    
    # Basic information
    print(f"🎯 Project: {config.project_name}/{config.experiment_name}")
    print(f"🤖 Model: {config.pretrained_model}")
    print(f"📊 Dataset: {config.data_yaml}")
    print(f"⚙️ Configuration:")
    print(f"   - Epochs: {config.epochs}")
    print(f"   - Batch Size: {config.batch_size}")
    print(f"   - Image Size: {config.image_size}")
    print(f"   - Device: {config.device}")
    
    # Training status
    if training_results:
        print(f"\n✅ Training Status: COMPLETED")
        
        # Validation metrics
        if validation_results and hasattr(validation_results, 'box'):
            box_metrics = validation_results.box
            print(f"\n📊 Final Metrics:")
            print(f"   - mAP@0.5: {box_metrics.map50:.4f}")
            print(f"   - mAP@0.5:0.95: {box_metrics.map:.4f}")
            print(f"   - Precision: {box_metrics.mp:.4f}")
            print(f"   - Recall: {box_metrics.mr:.4f}")
            
            # Performance evaluation
            if box_metrics.map50 >= 0.7:
                print(f"   🎉 EXCELLENT: Model meets target performance (mAP@0.5 ≥ 0.7)")
            elif box_metrics.map50 >= 0.5:
                print(f"   ✅ GOOD: Model shows good performance (mAP@0.5 ≥ 0.5)")
            else:
                print(f"   ⚠️ FAIR: Model needs improvement (mAP@0.5 < 0.5)")
    else:
        print(f"\n❌ Training Status: FAILED or INCOMPLETE")
    
    # File locations
    results_dir = Path(config.project_name) / config.experiment_name
    drive_dir = Path('/content/drive/MyDrive/insect_detection_training/results') / config.experiment_name
    
    print(f"\n📁 Output Locations:")
    print(f"   - Local: {results_dir}")
    print(f"   - Google Drive: {drive_dir}")
    
    # Next steps
    print(f"\n🚀 Next Steps:")
    print(f"   1. Download model files (best.pt) for deployment")
    print(f"   2. Test model on new images")
    print(f"   3. Deploy to production environment")
    print(f"   4. Monitor performance on real data")
    
    if training_results:
        print(f"\n💡 Optimization Tips:")
        if validation_results and hasattr(validation_results, 'box'):
            if validation_results.box.map50 < 0.7:
                print(f"   - Try training for more epochs")
                print(f"   - Increase model size (yolov8s or yolov8m)")
                print(f"   - Add more training data")
                print(f"   - Adjust data augmentation")
            else:
                print(f"   - Model performance is good!")
                print(f"   - Consider model compression for deployment")
                print(f"   - Test on edge devices (Raspberry Pi)")
    
    print("="*60)

# Generate summary
generate_training_summary(config, training_results, validation_results)

In [ ]:
# Usage examples for trained model
def show_usage_examples(config):
    print("💻 USAGE EXAMPLES")
    print("="*50)
    
    model_path = f"{config.project_name}/{config.experiment_name}/weights/best.pt"
    
    print("\n🐍 Python Usage:")
    print("```python")
    print("from ultralytics import YOLO")
    print("")
    print(f"# Load trained model")
    print(f"model = YOLO('{model_path}')")
    print("")
    print("# Run inference on single image")
    print("results = model('path/to/image.jpg')")
    print("")
    print("# Run inference on multiple images")
    print("results = model(['img1.jpg', 'img2.jpg'])")
    print("")
    print("# Save results with annotations")
    print("for r in results:")
    print("    r.save(filename='result.jpg')")
    print("```")
    
    print("\n🖥️ Command Line Usage:")
    print("```bash")
    print(f"# Single image prediction")
    print(f"yolo predict model={model_path} source=image.jpg")
    print("")
    print(f"# Batch prediction")
    print(f"yolo predict model={model_path} source=images_folder/")
    print("")
    print(f"# Video prediction")
    print(f"yolo predict model={model_path} source=video.mp4")
    print("```")
    
    print("\n🌐 Integration with detect_insect.py:")
    print("```bash")
    print(f"# Use trained model with detection script")
    print(f"python detect_insect.py \\")
    print(f"    --input input_images/ \\")
    print(f"    --output output_images/ \\")
    print(f"    --model {model_path}")
    print("```")
    
    print("\n📱 Export for Edge Deployment:")
    print("```python")
    print("# Export to ONNX for cross-platform deployment")
    print(f"model = YOLO('{model_path}')")
    print("model.export(format='onnx')")
    print("")
    print("# Export to TensorRT for NVIDIA GPUs")
    print("model.export(format='engine')")
    print("```")

# Show usage examples
if training_results:
    show_usage_examples(config)
else:
    print("⚠️ No usage examples available - training was not completed")

---

## 🎉 トレーニング完了！

**おめでとうございます！** 昆虫検出のためのYOLOv8トレーニングパイプラインが正常に完了しました。

### 📋 達成した内容:
- ✅ GPU加速トレーニング環境のセットアップ
- ✅ カブトムシ検出データセットの準備と検証
- ✅ カスタムYOLOv8モデルの訓練
- ✅ モデル性能の評価
- ✅ 展開用モデルのエクスポート
- ✅ 結果のGoogle Driveへの保存

### 🚀 次のステップ:
1. **訓練済みモデルのダウンロード** (`best.pt`) をローカルで使用
2. **新しいカブトムシ画像でのテスト**
3. **提供された使用例を使用した本番環境への展開**
4. **性能の監視** と必要に応じた再訓練

### 📚 リソース:
- [YOLOv8 ドキュメント](https://docs.ultralytics.com/)
- [モデル展開ガイド](https://docs.ultralytics.com/modes/export/)
- [性能最適化](https://docs.ultralytics.com/guides/model-optimization/)

---

*🐛 楽しいカブトムシ検出を！ 🐛*